In [1]:
from __future__ import print_function 
import os, sys
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf

sys.path.append('..')
from tfomics import layers, utils, init, learn
from tfomics import neuralnetwork as nn
from model_zoo import simple_genome_model

In [2]:

def load_data(filepath):
    # load training set
    print("loading data from: " + filepath)
    f = open(filepath, 'rb')
    print("loading train data")
    train = cPickle.load(f)
    print("loading cross-validation data")
    cross_validation = cPickle.load(f)
    print("loading test data")
    test = cPickle.load(f)
    f.close()

    X_train = train[0].transpose((0,2,1))
    y_train = train[1]
    X_valid = cross_validation[0].transpose((0,2,1))
    y_valid = cross_validation[1]
    X_test = test[0].transpose((0,2,1))
    y_test = test[1]
    
    X_train = X_train.reshape(X_train.shape + (1,)) 
    X_train = X_train.transpose([0,1,3,2]).astype(np.float32)
    y_train = y_train.astype(np.int32)

    X_valid = X_valid.reshape(X_valid.shape + (1,)) 
    X_valid = X_valid.transpose([0,1,3,2]).astype(np.float32)
    y_valid = y_valid.astype(np.int32)


    X_test = X_test.reshape(X_test.shape + (1,)) 
    X_test = X_test.transpose([0,1,3,2]).astype(np.float32)
    y_test = y_test.astype(np.int32)

    return X_train, y_train, X_valid, y_valid, X_test, y_test


# load data
filename = 'Localized_N=100000_S=200_M=50_G=20_data.pickle'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)
X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(filepath)

loading data from: /home/peter/Code/tensorflow/data/Localized_N=100000_S=200_M=50_G=20_data.pickle
loading train data
loading cross-validation data
loading test data


In [3]:
# get shapes
num_data, seq_length, _, dim = X_train.shape
input_shape=[None, seq_length, 1, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

# load model
net, placeholders, optimization = simple_genome_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders['inputs'])
nnmodel.inspect_layers()

# compile neural trainer
save_name = 'test'
save_path = utils.make_directory('results', 'simple_genome')
filepath = os.path.join(save_path, save_name)
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath=filepath)


----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 200, 1, 4)
layer2: conv1
(?, 200, 1, 25)
layer3: conv1_batch
(?, 200, 1, 25)
layer4: conv1_active
(?, 200, 1, 25)
layer5: conv1_pool
(?, 20, 1, 25)
layer6: resid1_1resid
(?, 20, 1, 25)
layer7: resid1_1resid_norm
(?, 20, 1, 25)
layer8: resid1_1resid_active
(?, 20, 1, 25)
layer9: resid1_2resid
(?, 20, 1, 25)
layer10: resid1_2resid_norm
(?, 20, 1, 25)
layer11: resid1_residual
(?, 20, 1, 25)
layer12: resid1_resid
(?, 20, 1, 25)
layer13: conv2
(?, 15, 1, 50)
layer14: conv2_batch
(?, 15, 1, 50)
layer15: conv2_active
(?, 15, 1, 50)
layer16: conv2_pool
(?, 3, 1, 50)
layer17: conv2_dropout
(?, 3, 1, 50)
layer18: dense1
(?, 20)
layer19: dense1_bias
(?, 20)
layer20: dense1_active
(?, 20)
layer21: output
(?, 20)
----------------------------------------------------------------------------
making directory: r

In [4]:
# run session
sess = tf.Session()

# initialize variables
sess.run(tf.initialize_all_variables())

In [5]:
X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(sess, nntrainer, data, batch_size=128, num_epochs=500, 
                        patience=10, verbose=0, shuffle=True)

  valid loss:		0.16027
  valid accuracy:	0.95211+/-0.02677
  valid auc-roc:	0.74303+/-0.13027
  valid auc-pr:		0.23430+/-0.23014
lower cross-validation found
saving model to:  results/simple_genome/test_best.ckpt
  valid loss:		0.12542
  valid accuracy:	0.96000+/-0.02461
  valid auc-roc:	0.83833+/-0.11724
  valid auc-pr:		0.39409+/-0.32730
lower cross-validation found
saving model to:  results/simple_genome/test_best.ckpt
  valid loss:		0.09489
  valid accuracy:	0.96787+/-0.02016
  valid auc-roc:	0.90831+/-0.09582
  valid auc-pr:		0.54771+/-0.32471
lower cross-validation found
saving model to:  results/simple_genome/test_best.ckpt
  valid loss:		0.07030
  valid accuracy:	0.97566+/-0.01636
  valid auc-roc:	0.94478+/-0.07905
  valid auc-pr:		0.68378+/-0.31480
lower cross-validation found
saving model to:  results/simple_genome/test_best.ckpt
  valid loss:		0.05535
  valid accuracy:	0.98057+/-0.01472
  valid auc-roc:	0.96275+/-0.05863
  valid auc-pr:		0.75592+/-0.29986
lower cross-validat

KeyboardInterrupt: 

In [6]:
X = {'inputs': X_test, 'targets': y_test, 'keep_prob': 1, 'is_training': False}
nntrainer.test_model(sess, X, batch_size=512)

  test  loss:		0.02098
  test  accuracy:	0.99330+/-0.00520
  test  auc-roc:	0.99667+/-0.00375
  test  auc-pr:		0.92824+/-0.14988


0.020981990856428941